<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [36]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [37]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [38]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [39]:
%%sql
SELECT * FROM Movie

 * sqlite:///movies.sqlite
Done.


Title,Production_year,IMDB_key,Running_time
The House That Jack Built,2018,tt4003440,02:32:00
Melancholia,2011,tt1527186,02:15:00
Antichrist,2009,tt0870984,01:48:00
Dogville,2003,tt0276919,02:58:00


+ Show the performance dates for one of the movies.

In [62]:
%%sql
SELECT * FROM Screening

 * sqlite:///movies.sqlite
Done.


IMDB_key,Screening_ID,Theater_name,Start_time,Seats
tt0870984,1,Kino der Toten,2023-02-14 19:00:00,100


+ Show all data concerning performances at a given theatere
  on a given date.

In [63]:
%%sql
SELECT * FROM Screening
WHERE Start_time LIKE '2023-02-14%'

 * sqlite:///movies.sqlite
Done.


IMDB_key,Screening_ID,Theater_name,Start_time,Seats
tt0870984,1,Kino der Toten,2023-02-14 19:00:00,100


+ List all customers

In [64]:
%%sql
SELECT * FROM User

 * sqlite:///movies.sqlite
Done.


User_name,Full_name,Password
hannus,Hannes Östergren,password
anotherUser,John Doe,seven


+ List all tickets

In [65]:
%%sql

SELECT * FROM Ticket

 * sqlite:///movies.sqlite
Done.


User_name,Screening_ID,Ticket_ID
hannus,1,80bc8e288113f5f8ad96a74777f32ece


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [66]:
%%sql
INSERT INTO Ticket(User_name, Screening_ID)
VALUES ('anotherUser', '1')

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [67]:
%%sql
SELECT Ticket_ID
FROM Ticket
WHERE rowid = last_insert_rowid()

 * sqlite:///movies.sqlite
Done.


Ticket_ID
66512f283e811261f155533a5100b0ec


+ Try to insert two movie theaters with the same name (this
  should fail).

In [68]:
%%sql
INSERT INTO Theater(Name, Capacity)
VALUES ('Kino der Toten', '2000')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) UNIQUE constraint failed: Theater.Name
[SQL: INSERT INTO Theater(Name, Capacity)
VALUES ('Kino der Toten', '2000')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [69]:
%%sql
INSERT INTO Screening(IMDB_key, Screening_ID, Theater_name, Start_time, Seats)
VALUES ('blabla', '2', 'Le Gamaar', '20:00:00', '400')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO Screening(IMDB_key, Screening_ID, Theater_name, Start_time, Seats)
VALUES ('blabla', '2', 'Le Gamaar', '20:00:00', '400')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [71]:
%%sql
INSERT INTO Ticket(User_name, Screening_ID)
--VALUES ('nonExistentUser', '1')
VALUES ('anotherUser', '2')

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO Ticket(User_name, Screening_ID)
--VALUES ('nonExistentUser', '1')
VALUES ('anotherUser', '2')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
